In [62]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import cv2
import numpy as np

from PIL import Image, ImageDraw

In [7]:
BASE_DIR = "D:\\_hack\\2021-22\\d-hacks-ai.ru\\758258-Tatarstan\\data_for_yolo\\data"
train_images_dir = os.path.join(BASE_DIR, "images", "train")
train_orig_images_dir = os.path.join(BASE_DIR, "images", "train-original")
train_labels_dir = os.path.join(BASE_DIR, "labels", "train")
test_images_dir = os.path.join(BASE_DIR, "images", "test")

In [8]:
len(os.listdir(train_orig_images_dir))

2002

## Посмотрим на картинку

In [122]:
image_name = random.choice(os.listdir(train_images_dir))
# image_name = "3.jpg"
image = cv2.imread(os.path.join(train_images_dir, image_name))
im_height, im_width, channels = image.shape

image_filename = str(image_name).split(".")[0]
label_path = os.path.join(train_labels_dir, image_filename)  + ".txt"
with open(label_path, 'r') as f:
  data = f.read()
  data = [i for i in data.split('\n') if i != '']

for line in data:
  val = [float(i) for i in line.split()]
  cls, xywh = val[0], val[1:5]
  center_x, center_y, width, height = xywh
  xmin = int((center_x - (width / 2)) * im_width)
  xmax = int((center_x + (width / 2)) * im_width)
  ymin = int((center_y - (height / 2)) * im_height)
  ymax = int((center_y + (height / 2)) * im_height)
  print(f'cls: {cls}, x_min: {xmin}, y_min: {ymin}, x_max: {xmax}, y_max: {ymax}')

  cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0,0,255), 3)

# You may need to convert the color.
img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
im_pil = Image.fromarray(img)
im_pil.show()


cls: 0.0, x_min: 219, y_min: 196, x_max: 515, y_max: 328
cls: 0.0, x_min: 1164, y_min: 200, x_max: 1334, y_max: 297
cls: 1.0, x_min: 926, y_min: 113, x_max: 938, y_max: 125
cls: 1.0, x_min: 964, y_min: 113, x_max: 974, y_max: 123
cls: 3.0, x_min: 962, y_min: 115, x_max: 978, y_max: 152
cls: 3.0, x_min: 921, y_min: 114, x_max: 942, y_max: 155


# Удалим все размеченные участки изображения

In [123]:
image_fon = cv2.imread(os.path.join(train_images_dir, image_name))
fon_height, fon_width, channels = image_fon.shape

for line in data:
  val = [float(i) for i in line.split()]
  cls, xywh = val[0], val[1:5]
  center_x, center_y, width, height = xywh
  xmin = int((center_x - (width / 2)) * im_width)
  xmax = int((center_x + (width / 2)) * im_width)
  ymin = int((center_y - (height / 2)) * im_height)
  ymax = int((center_y + (height / 2)) * im_height)

  # закрашиваем разноцветными квадратиками
  rec_width = 15
  for rec_x1 in range(xmin, xmax, rec_width):
    for rec_y1 in range(ymin, ymax, rec_width):
        r = random.randint(1,254)
        g = random.randint(1,254)
        b = random.randint(1,254)

        cv2.rectangle(image_fon, (rec_x1, rec_y1), (rec_x1+rec_width, rec_y1+rec_width), (r,g,b), int(rec_width/2))

img = cv2.cvtColor(image_fon, cv2.COLOR_BGR2RGB)
fon_im_pil = Image.fromarray(img)
fon_im_pil.show()


# Возьмем другое изображение и скопируем случайный обьект на фон

In [70]:
image_name = random.choice(os.listdir(train_images_dir))
image_obj = cv2.imread(os.path.join(train_images_dir, image_name))
im_height, im_width, channels = image_obj.shape

image_filename = str(image_name).split(".")[0]
label_path = os.path.join(train_labels_dir, image_filename)  + ".txt"
with open(label_path, 'r') as f:
  data = f.read()
  data = [i for i in data.split('\n') if i != '']

obj_line = random.choice(data)

val = [float(i) for i in obj_line.split()]
cls, xywh = val[0], val[1:5]
center_x, center_y, width, height = xywh
xmin = int((center_x - (width / 2)) * im_width)
xmax = int((center_x + (width / 2)) * im_width)
ymin = int((center_y - (height / 2)) * im_height)
ymax = int((center_y + (height / 2)) * im_height)
print(f'cls: {cls}, x_min: {xmin}, y_min: {ymin}, x_max: {xmax}, y_max: {ymax}')

cv2.rectangle(image_obj, (xmin, ymin), (xmax, ymax), (255,0,0), 2)

img = cv2.cvtColor(image_obj, cv2.COLOR_BGR2RGB)
im_pil = Image.fromarray(img)
im_pil.show()


cls: 3.0, x_min: 815, y_min: 743, x_max: 911, y_max: 1033


In [71]:
crop_img = image_obj[ymin:ymax, xmin:xmax]
obj_height, obj_width, channels = crop_img.shape

img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
crop_im_pil = Image.fromarray(img)
crop_im_pil.show()


# Переносим на фон

In [72]:
new_x = random.randint(5, fon_width - obj_width - 5)
new_y = random.randint(5, fon_height - obj_height - 5)

fon_im_pil.paste(crop_im_pil, (new_x, new_y))

fon_im_pil.show()


# пересчитаем положение label

In [73]:
center_x = (new_x + obj_width / 2) / fon_width
center_y = (new_y + obj_height / 2) / fon_height
width = obj_width / fon_width
height = obj_height / fon_height


In [74]:
# Проверим пересчет

xmin = int((center_x - (width / 2)) * fon_width)
xmax = int((center_x + (width / 2)) * fon_width)
ymin = int((center_y - (height / 2)) * fon_height)
ymax = int((center_y + (height / 2)) * fon_height)
print(f'cls: {cls}, x_min: {xmin}, y_min: {ymin}, x_max: {xmax}, y_max: {ymax}')

img1 = ImageDraw.Draw(fon_im_pil) 
img1.rectangle([(xmin, ymin), (xmax, ymax)], outline ="red")
fon_im_pil.show()


cls: 3.0, x_min: 1558, y_min: 886, x_max: 1654, y_max: 1177


# сгенерируем 5000 картинок

In [149]:
def get_fon():
    fon_image_name = random.choice(os.listdir(train_images_dir))
    image_fon = cv2.imread(os.path.join(train_images_dir, fon_image_name))
    fon_height, fon_width, channels = image_fon.shape

    image_filename = str(fon_image_name).split(".")[0]
    label_path = os.path.join(train_labels_dir, image_filename)  + ".txt"
    with open(label_path, 'r') as f:
        data = f.read()
    data = [i for i in data.split('\n') if i != '']

    # закрашиваем разноцветными квадратиками
    for line in data:
        val = [float(i) for i in line.split()]
        cls, xywh = val[0], val[1:5]
        center_x, center_y, width, height = xywh
        xmin = int((center_x - (width / 2)) * fon_width)
        xmax = int((center_x + (width / 2)) * fon_width)
        ymin = int((center_y - (height / 2)) * fon_height)
        ymax = int((center_y + (height / 2)) * fon_height)

        # cv2.rectangle(image_fon, (xmin, ymin), (xmax, ymax), (0,0,255), 3)

        rec_width = 5
        for rec_x1 in range(xmin, xmax, rec_width):
            for rec_y1 in range(ymin, ymax, rec_width):
                r = random.randint(1,254)
                g = random.randint(1,254)
                b = random.randint(1,254)

                cv2.rectangle(image_fon, (rec_x1, rec_y1), (rec_x1+rec_width, rec_y1+rec_width), (r,g,b), int(rec_width/2))    

    img = cv2.cvtColor(image_fon, cv2.COLOR_BGR2RGB)
    fon_im_pil = Image.fromarray(img)  

    return fon_im_pil




In [162]:
def get_obj():
    # обьект
    image_name_obj = random.choice(os.listdir(train_images_dir))
    image_obj = cv2.imread(os.path.join(train_images_dir, image_name_obj))
    obj_im_height, obj_im_width, channels = image_obj.shape

    image_filename = str(image_name_obj).split(".")[0]
    label_path = os.path.join(train_labels_dir, image_filename)  + ".txt"
    with open(label_path, 'r') as f:
        data = f.read()
    data = [i for i in data.split('\n') if i != '']

    obj_line = random.choice(data)

    val = [float(i) for i in obj_line.split()]
    cls, xywh = val[0], val[1:5]
    center_x, center_y, width, height = xywh
    xmin = int((center_x - (width / 2)) * obj_im_width)
    xmax = int((center_x + (width / 2)) * obj_im_width)
    ymin = int((center_y - (height / 2)) * obj_im_height)
    ymax = int((center_y + (height / 2)) * obj_im_height)
    # print(f'obj: cls: {cls}, x_min: {xmin}, y_min: {ymin}, x_max: {xmax}, y_max: {ymax}')

    crop_img = image_obj[ymin:ymax, xmin:xmax]
    obj_height, obj_width, channels = crop_img.shape
    img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
    crop_im_pil = Image.fromarray(img)    

    return crop_im_pil

crop_im_pil = get_obj()
crop_im_pil.show()

In [164]:
for i in range(1, 5000):

    try:
        fon_im_pil = get_fon()
        fon_width, fon_height = fon_im_pil.size

        crop_im_pil = get_obj()            
        obj_width, obj_height = crop_im_pil.size

        # переносим на фон
        new_x = random.randint(5, fon_width - obj_width - 5)
        new_y = random.randint(5, fon_height - obj_height - 5)
        fon_im_pil.paste(crop_im_pil, (new_x, new_y))

        center_x = (new_x + obj_width / 2) / fon_width
        center_y = (new_y + obj_height / 2) / fon_height
        width = obj_width / fon_width
        height = obj_height / fon_height


        # сохраняем
        new_image_path = os.path.join(train_images_dir, str(i))  + ".jpg"
        fon_im_pil.save(new_image_path)

        new_label_path = os.path.join(train_labels_dir, str(i))  + ".txt"
        new_line = f"{int(cls)} {center_x} {center_y} {width} {height}"
        with open(new_label_path, 'w') as f:
            f.write(new_line)
        f.close()
    except:
        pass

    